In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction\\notebook'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.heartstrokeprediction.constants import *
from src.heartstrokeprediction.utils.common import read_yaml, create_directories

[2025-04-09 02:39:04,435: INFO: __init__: This is an info message.]
[2025-04-09 02:39:04,437: ERROR: __init__: This is an error message.]


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [12]:
import os
from src.heartstrokeprediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [13]:
class DataTransformation: 
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        # Load the dataset
        data = pd.read_csv(self.config.data_path)

        # Drop the 'id' column from the dataset if it exists
        if 'id' in data.columns:
            data.drop(columns=['id'], inplace=True)
            logger.info("Dropped 'id' column from the dataset")

        # Calculate the mean value of the 'bmi' column and fill NaN values with the mean
        mean_value = data['bmi'].mean()
        data['bmi'].fillna(mean_value, inplace=True)

        # Apply label encoding to categorical columns
        label_encoder = LabelEncoder()
        data['gender'] = label_encoder.fit_transform(data['gender'])
        data['ever_married'] = label_encoder.fit_transform(data['ever_married'])
        data['work_type'] = label_encoder.fit_transform(data['work_type'])
        data['Residence_type'] = label_encoder.fit_transform(data['Residence_type'])
        data['smoking_status'] = label_encoder.fit_transform(data['smoking_status'])
        
        logger.info("Applied label encoding to categorical columns")

        # Split the data into features and target (assuming the target is the last column)
        X = data.drop(columns=['stroke'])
        y = data['stroke']

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Save the transformed data to CSV files
        train_transformed = pd.concat([X_train, y_train], axis=1)
        test_transformed = pd.concat([X_test, y_test], axis=1)
        
        train_transformed.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_transformed.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted and transformed data into training and test sets")
        logger.info(f"Train shape: {train_transformed.shape}")
        logger.info(f"Test shape: {test_transformed.shape}")

        print(f"Train shape: {train_transformed.shape}")
        print(f"Test shape: {test_transformed.shape}")
        print(f"Train target column 'stroke' has {y_train.isnull().sum()} NaN values.")
        print(f"Test target column 'stroke' has {y_test.isnull().sum()} NaN values.")

In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-04-09 02:49:08,519: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-09 02:49:08,522: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-09 02:49:08,526: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-09 02:49:08,530: INFO: common: created directory at: artifacts]
[2025-04-09 02:49:08,532: INFO: common: created directory at: artifacts/data_transformation]
[2025-04-09 02:49:08,545: INFO: 3276256660: Dropped 'id' column from the dataset]
[2025-04-09 02:49:08,555: INFO: 3276256660: Applied label encoding to categorical columns]
[2025-04-09 02:49:08,584: INFO: 3276256660: Splitted and transformed data into training and test sets]
[2025-04-09 02:49:08,585: INFO: 3276256660: Train shape: (3577, 11)]
[2025-04-09 02:49:08,586: INFO: 3276256660: Test shape: (1533, 11)]
Train shape: (3577, 11)
Test shape: (1533, 11)
Train target column 'stroke' has 0 NaN values.
Test target column 'stroke' has 0 NaN values.


C:\Users\kk061\AppData\Local\Temp\ipykernel_6784\3276256660.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['bmi'].fillna(mean_value, inplace=True)
